## Load BOM

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import time as t
import json

BOM_path = f"../DAC_GainOutputStage7.csv"

df = pd.read_csv(BOM_path, delimiter = ";")
# display(df)
print("Done")

Done


## Search Parts

In [2]:
# from dotenv import load_dotenv
# load_dotenv()
MOUSER_API_KEY = "c7b47461-c777-4dbc-8413-9bb69528b78f"

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = {
    'apiKey': MOUSER_API_KEY,
}

json_data = {
    'SearchByPartRequest': {
        'mouserPartNumber': 'LM317',
        'partSearchOptions': 'string',
        'mouserPaysCustomsAndDuties': False,
    },
}

response = requests.post('https://api.mouser.com/api/v1/search/partnumber', params=params, headers=headers, json=json_data)

print("Done")

Done


## Filter Response

In [20]:
if response.status_code == 200:
    data = response.json()
else:
    # Handle errors
    print(f'Error: {response.status_code} - {response.text}')

df = pd.DataFrame()

# Sort and append new data
if data == {}:
    pass
else:
    for part in data["SearchResults"]["Parts"]:
        df = pd.concat([df, pd.json_normalize(part)], ignore_index=True)

df.drop(["ImagePath",
         "AlternatePackagings",
         "ProductDetailUrl",
         "Reeling",
         "MultiSimBlue",
         "InfoMessages",
         "SurchargeMessages",
         "ProductCompliance",
         "REACH-SVHC",
         "RestrictionMessage",
         "ROHSStatus",
         "ProductAttributes",
         "FactoryStock",
         "AvailabilityOnOrder",
        ], axis = 1, inplace = True)

df_exploded = df.explode("PriceBreaks").reset_index(drop=True)
pricing_df = pd.json_normalize(df_exploded["PriceBreaks"])
df_combined = pd.concat([df_exploded.drop(columns=["PriceBreaks"]), pricing_df], axis=1)
df = df_combined.set_index(["MouserPartNumber", "Quantity"]).sort_index()

for column in df.columns:
    try:
        df[column] = pd.to_numeric(df[column])
    except Exception as e:
        pass

df = df[df["AvailabilityInStock"] > 0]
df = df[df["Min"] == 1]
df = df[df["Mult"] == 1]

df =  df[df.index.isin([1.0], level=1)]

display(df)

,,Availability,DataSheetUrl,Description,Category,LeadTime,LifecycleStatus,Manufacturer,ManufacturerPartNumber,Min,Mult,SuggestedReplacement,AvailabilityInStock,UnitWeightKg.UnitWeight,Price,Currency
MouserPartNumber,Quantity,,,,,,,,,,,,,,,
511-LM317BT,1.0,3593 In Stock,,Linear Voltage Regulators 1.2 V to 37 V adjust...,Linear Voltage Regulators,112 Days,None,STMicroelectronics,LM317BT,1,1,,3593.0,0.001356,$0.62,USD
511-LM317D2T-TR,1.0,7965 In Stock,,Linear Voltage Regulators 1.2-37V Adj Positive,Linear Voltage Regulators,112 Days,None,STMicroelectronics,LM317D2T-TR,1,1,,7965.0,0.002000,$1.13,USD
511-LM317P,1.0,1825 In Stock,,Linear Voltage Regulators 1.2-37V Adj Positive,Linear Voltage Regulators,112 Days,None,STMicroelectronics,LM317P,1,1,,1825.0,0.002300,$1.24,USD
511-LM317T,1.0,20509 In Stock,,Linear Voltage Regulators 1.2-37V Adj Positive...,Linear Voltage Regulators,112 Days,None,STMicroelectronics,LM317T,1,1,,20509.0,0.002300,$0.56,USD
511-LM317T-DG,1.0,1990 In Stock,,Linear Voltage Regulators Dual Guage ADJ 1.2V ...,Linear Voltage Regulators,112 Days,None,STMicroelectronics,LM317T-DG,1,1,,1990.0,0.002300,$0.83,USD
595-LM317DCY,1.0,15794 In Stock,,Linear Voltage Regulators 3 T 1.5A Adj Pos V A...,Linear Voltage Regulators,42 Days,None,Texas Instruments,LM317DCY,1,1,595-LM317DCYR,15794.0,0.000112,$1.17,USD
595-LM317DCYG3,1.0,1048 In Stock,,Linear Voltage Regulators 3Pin 1.5A Adj Vltg R...,Linear Voltage Regulators,42 Days,None,Texas Instruments,LM317DCYG3,1,1,595-LM317DCY,1048.0,0.000112,$1.17,USD
595-LM317DCYR,1.0,24080 In Stock,,Linear Voltage Regulators 3 Term Adj. Pos. A 5...,Linear Voltage Regulators,84 Days,None,Texas Instruments,LM317DCYR,1,1,595-LM317DCY,24080.0,0.000330,$0.75,USD
595-LM317DCYRG3,1.0,11580 In Stock,,Linear Voltage Regulators 3Pin 1.5A Adj Vltg R...,Linear Voltage Regulators,84 Days,None,Texas Instruments,LM317DCYRG3,1,1,595-LM317DCYR,11580.0,0.000110,$1.68,USD
